# Lab-8: Deep Learning with Lightning

This lab is design for investigating the basic architecture and parameter searching techniques in deep learning literature. We use a fundamental dataset, MNIST, with the help of PyTorch Lightning and design our model to handle image classification task.

### General Announcements

* The exercises on this sheet are graded by a maximum of **14 points**. You will be asked to implement several functions.
* Team work is not allowed! Everybody implements his/her own code. Discussing issues with others is fine, sharing code with others is not. 
* If you use any code fragments found on the Internet, make sure you reference them properly.
* You can send your questions via email to the TAs until the deadline.

### Suggestions

Please install pytorch lightning packages via conda or pip before starting the lab session. You can use the tutorials of [Pytorch Lightning](https://lightning.ai/docs/pytorch/stable/notebooks/lightning_examples/mnist-hello-world.html).

Please also check: [Tensorboard](https://pytorch.org/tutorials/recipes/recipes/tensorboard_with_pytorch.html)

For installing lightning: [Link](https://lightning.ai/pytorch-lightning)

In [1]:
# Basic Machine Learning Modules
import pandas
import numpy
import sklearn

# Deep Learning Modules
import torch
import lightning.pytorch as pl

# Visualization Modules
import matplotlib.pyplot as plt

# Others
from torchvision.datasets import MNIST
from torchvision import transforms
from torchmetrics import Accuracy
from lightning.pytorch.loggers import CSVLogger, TensorBoardLogger
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import TQDMProgressBar
import warnings
warnings.filterwarnings("ignore")


DATASET_PATH = '~/Desktop/MNIST/'  # You can change them
EXPERIMENTS_PATH = '~/Desktop/MNIST_EXP'  # You can change them

# 1) Dataset (3 Points)

- Design a DataModule for MNIST.
- Use 80/20 % split for train/val sets.

In [ ]:
class MNISTDataModule(pl.LightningDataModule):
    def __init__(self, data_folder: str = DATASET_PATH, batch_size: int = 64, num_cpu: int = 1):
        super().__init__()
        self.path = data_folder
        self.batch_size = batch_size
        self.num_cpu = num_cpu
        self.transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
    
    def prepare_data(self) -> None:
        # Download MNIST Data in self.path
    
    def setup(self, stage: str = 'fit') -> None:
        if stage in ['fit', 'tune']:
            self.train_dataset = ...  # Insert your code 
        
        if stage in ['fit', 'tune', 'validate']:
            self.val_dataset = ...  # Insert your code 
        
        elif stage in ['test', 'predict']:
            self.test_dataset = ...  # Insert your code 
        
        else:
            raise NotImplementedError('Unknown Stage: {}'.format(stage))
            
    def train_dataloader(self) -> torch.utils.data.DataLoader:
        return torch.utils.data.DataLoader(
            batch_size=self.batch_size, num_workers=self.num_cpu,  # DO NOT CHANGE IN VAL/TEST LOADERS
            dataset=self.train_dataset, shuffle=True,  # Could be changed in val/test loaders
        )
    def val_dataloader(self) -> torch.utils.data.DataLoader:
        # Insert your code
    
    def test_dataloader(self) -> torch.utils.data.DataLoader:
        # Insert your code
    
    def predict_dataloader(self) -> torch.utils.data.DataLoader:
        return test_dataloader()

data_module = MNISTDataModule()
data_module.prepare_data()
data_module.setup('fit')
dl = data_module.train_dataloader()
print(next(dl.__iter__()))

In [ ]:
# Do not change! Only for checking
print('Shape of Images: [B x C x H x W] = ', next(dl.__iter__())[0].shape)
print('Shape of Labels: [B] = ', next(dl.__iter__())[1].shape)

## 2) Neural Network Architecture (2 Points)

- Design an model with 3 Convolutional layers and 1 fully-connected layer, in this order:
    - Convolution: Kernel size = 3x3, padding = 'same', number of filters = 4
    - Convolution: Kernel size = 3x3, padding = 'same', number of filters = 8
    - Convolution: Kernel size = 3x3, padding = 'same', number of filters = 4
    - Linear: No Bias, num_class = 10 in MNIST Dataset
- Use rectified linear unit (ReLU) for activation function. 
- Initialize all the weights with `xavier_uniform`.

In [ ]:
class RawModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # Insert your code
        self.model = ...
        self.model.apply(self.initialize_weights)
    
    @staticmethod
    def initialize_weights(module: torch.nn.Module) -> None:
        # Insert your code
        
    def forward(self, images: torch.Tensor) -> torch.Tensor:
        """
        Input: torch.Tensor | dtype=torch.float | shape=[B, C, H, W]
        Output: torch.Tensor | dtype=torch.float | shape=[B, num_class]
        """
        # Insert your code

model = RawModel()
with torch.no_grad():
    sample_image = torch.rand(size=(4, 1, 28, 28))
    output = model(sample_image)
    print(output.shape, output.dtype)


## 3) Experiments (4 Points)

- Define training/validation/test step and optimizer with cross-entropy loss and Adam optimizer.
- Use accuracy scores for monitoring the experiment. (multiclass accuracy from Lightning Metrics)

In [ ]:
class MNISTExperiment(pl.LightningModule):
    def __init__(self, learning_rate: float = 1e-3):
        super().__init__()
        self.model = RawModel()
        self.learning_rate = learning_rate
        
        self.train_scores = ...  # Insert your code
        self.validation_scores = ...  # Insert your code
        self.test_scores = ...  # Insert your code

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)

    def training_step(self, batch, batch_idx) -> torch.Tensor:
        x, y = batch
        y_hat = self(x)
        
        loss = ...  # Insert your code
        # Insert your code: Score Calculation
        
        self.log('train_loss', loss)
        self.log('train_accuracy', ...)
        return loss
    
    def validation_step(self, batch, batch_idx) -> None:
        # Insert your code
        self.log('validation_accuracy', ...)
    
    def test_step(self, batch, batch_idx) -> None:
        # Insert your code
        self.log('test_accuracy', ...)

    def configure_optimizers(self):
        optimizer = ...  # Insert your code 
        return optimizer

experiment = MNISTExperiment()

- Define a trainer of lightning.
    - Maximum epoch = 20
    - accelerator = 'auto'
    - Use `CSVLogger` and `TensorBoardLogger`
    - Use `EarlyStopping` with patience epoch = 3
    - Use `TQDMProgressBar` with refresh rate = 10

In [ ]:
trainer = ...  # Insert your code

## 4) Results (4 Points)
- Train and test the `RawModel` and plot the score and loss values versus epoch.

In [ ]:
# Insert your code

- Re-design the model with dropout layer in-between the convolutional layers and re-train the model. as like `RawModel` and create a new module as `ModelWithDropout`. Try:
    - dropout probability = 0.1
    - dropout probability = 0.5
    - dropout probability = 0.9

In [ ]:
# Insert your code

- Re-design the model with batchnorm layer in-between the convolutional layers and re-train the model.

In [ ]:
# Insert your code

## 5) Conclusion (1 Point)
Comment on your findings:
- Show the results in a table via Pandas
- Which method is better? Why?
- Are the results significant? If not, how can we get significant ones?

ANSWER: ...